In [ ]:
%run ../code/init_mooc_nb.py

from matplotlib.colors import hsv_to_rgb
import scipy
from matplotlib import cm
from matplotlib import gridspec   

sigma0 = np.array([[1, 0], [0, 1]])
sigmax = np.array([[0, 1], [1, 0]])
sigmay = np.array([[0, -1j], [1j, 0]])
sigmaz = np.array([[1, 0], [0, -1]])

# dispersion functions
def plot_2D(X,Y,Z, ax_in=None):
    if ax_in==None:
        fig = plt.figure(figsize=(7,5))
        ax = fig.add_subplot(111, projection='3d')
    else:
        ax = ax_in

    vmin = np.array(Z).min()
    vmax = np.array(Z).max()
    
    if len(np.shape(Z)) > 2:
        for z in Z:
            ax.plot_surface(X, Y, z, rstride=1, cstride=1, cmap=cm.RdBu_r, 
                            linewidth=0.1, vmin=vmin, vmax=vmax)
    else:
        ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.RdBu_r, 
                        linewidth=0.1, vmin=vmin, vmax=vmax)

    if ax_in==None:
        return fig, ax
    else:
        return ax 
    

def evaluate_on_grid(X, Y, func):
    """ X, Y should be in np.meshgrid form. It's enough for func to work on floats. """
    data = []
    for xx, yy in zip(X, Y):
        row = []
        for i,j in zip(xx, yy):
            row.append(func(i,j))
        data.append(row)
    data = np.array(data)
    temp = np.shape(data)[2]
    try:
        data = [np.array(data[:,:,i]) for i in [temp/2-2, temp/2-1, temp/2, temp/2+1]]
    except:
        data = [np.array(data[:,:,i]) for i in [temp/2-1, temp/2]]
    return data


def diagonalize(sys, par):
    mat = sys.hamiltonian_submatrix(args=[par])
    ev, evec = scipy.linalg.eigh(mat)  # Automatically sorted
    return ev, evec


def dispersion_func(sys, par):
    def func(kx, ky):
        par.kx = kx
        par.ky = ky
        return diagonalize(sys, par)[0]
    return func


def plot_dispersion(sys, par, kx=[-np.pi, np.pi], ky=[-np.pi, np.pi], ax=None):

    Kx = np.linspace(kx[0], kx[1], 51)
    Ky = np.linspace(ky[0], ky[1], 51)
    mesh = np.meshgrid(Kx, Ky)
    energies = evaluate_on_grid(*mesh, func=dispersion_func(sys, par))
    
    if ax is None:
        fig, ax = plot_2D(*mesh, Z=energies)
        return fig, ax
    else:
        plot_2D(*mesh, Z=energies, ax_in=ax)


def d_wave_ribbon(w=10, direction='topo'):
    def hopx(site1, site2, par):
        return -par.t * sigmaz - par.delta * sigmax
    
    def hopy(site1, site2, par):
        return -par.t * sigmaz + par.delta * sigmax
    
    def onsite(site, par):
        return (4 * par.t - par.mu) * sigmaz 

    if direction == 'topo':
        def ribbon_shape(pos):
            (x, y) = pos
            return (0 <= y - x < w)
        sym = kwant.TranslationalSymmetry((1, 1))  
    else:
        def ribbon_shape(pos):
            (x, y) = pos
            return (0 <= y < w)
        sym = kwant.TranslationalSymmetry((1, 0))  
        
    
    lat = kwant.lattice.square()
    sys = kwant.Builder(sym)
    
    sys[lat.shape(ribbon_shape, (0, 0))] = onsite
    sys[kwant.HoppingKind((1,0), lat)] = hopx
    sys[kwant.HoppingKind((0,1), lat)] = hopy
    
    return sys.finalized()


#systems
def d_wave_infinite():
    def hopx(site1, site2, par):
        return -par.t * sigmaz - par.delta * sigmax
    
    def hopy(site1, site2, par):
        return -par.t * sigmaz + par.delta * sigmax
    
    def onsite(site, par):
        return (4 * par.t - par.mu) * sigmaz 

    def func(site, par):
        kx,ky = par.kx, par.ky
        H0, H1, H2 = onsite(site, par), hopx(site, site, par), hopy(site, site, par)
        
        return H0  \
            + H1*np.exp(-1j*kx) + H1.transpose().conjugate()*np.exp(+1j*kx) \
            + H2*np.exp(-1j*ky) + H2.transpose().conjugate()*np.exp(+1j*ky)
    
    lat = kwant.lattice.square()
    sys = kwant.Builder()
    sys[lat(0, 0)] = func
    return sys.finalized()

# Check out http://nbviewer.ipython.org/github/empet/Math/blob/master/DomainColoring.ipynb
# for a nice tutorial on visualizing complex functions.

def d_wave_offdiag(sys, args, steps=30):
    kxs, kys = np.mgrid[-np.pi:np.pi:1j*steps, -np.pi:np.pi:1j*steps]
     
    dets = np.zeros_like(kxs, dtype=complex)

    for i in range(steps):
        for j in range(steps):
            par.kx, par.ky = kxs[i, j], kys[i, j]
            ham = sys.hamiltonian_submatrix(args)
            #bring the chiral symmetric Hamiltonian in offdiagonal form
            u = (sigma0 + 1j * sigmax) / np.sqrt(2)
            ham = u.dot(ham).dot(u.T.conjugate())
            dets[i, j] = ham[1, 0]
    
    return dets

   
def graphene_infinite():
    def hop(site1, site2, par):
        temp = (par.t_1 * np.exp(1j * par.kx) + 
                par.t_23 * np.exp(0.5j * (np.sqrt(3) * par.ky - par.kx)) + 
                par.t_23 * np.exp(-0.5j * (np.sqrt(3) * par.ky + par.kx)))
        return temp

    lat = kwant.lattice.chain()
    sys = kwant.Builder()
    
    sys[lat(0)] = 0
    sys[lat(1)] = 0
    sys[kwant.HoppingKind((1,), lat)] = hop
                                                 
    return sys.finalized()


def graphene_offdiag(sys, args, steps=30):
    
    kxs, kys = np.mgrid[-np.pi:np.pi:1j*steps, -np.pi:np.pi:1j*steps]
    dets = np.zeros_like(kxs, dtype=complex) 
    
    for i in range(steps):
        for j in range(steps):
            par.kx, par.ky = kxs[i, j], kys[i, j]
            ham = sys.hamiltonian_submatrix(args)
            dets[i, j] = ham[1, 0]
    
    return dets


def Weyl_slab(w=5):
    def hopy(site1, site2, par):
        return 1j * par.t * sigmay - par.t * sigmaz
    
    def onsite(site, par):
        return (((2 + 2 * (1 - np.cos(par.kx)) + 2 * (1 - np.cos(par.ky))) * par.t - par.mu) * sigmaz +
                np.sin(par.kx) * par.t * sigmax)

    lat = kwant.lattice.chain()
    sys = kwant.Builder()
       
    def shape(pos):
        (x,) = pos
        return (0 <= x < w) 

    sys[lat.shape(shape, (0,))] = onsite
    sys[kwant.HoppingKind((1,), lat)] = hopy
    
    return sys.finalized() 

# function to set labels etc. for some standard plots
def plot_format_deth(ax):
    ax.set_xlim([-np.pi, np.pi])
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels(['$-\pi$', '$0$', '$\pi$'])
    ax.set_xlabel('$k_x$')
    ax.set_ylim([-np.pi, np.pi])
    ax.set_yticks([-np.pi, 0, np.pi])
    ax.set_yticklabels(['$-\pi$', '$0$', '$\pi$'])
    ax.set_ylabel('$k_y$')
    ax.set_title('$\det(h)$')
    
def plot_dets(dets):      
    
    H = np.angle(dets) / (2 * np.pi)
    V = np.abs(dets)
    H = np.mod(H,1)
    V /= np.max(V)
    V = 1 - V**2
    S = np.ones_like(H)

    HSV = np.dstack((H,S,V))
    RGB = hsv_to_rgb(HSV)
    plt.imshow(RGB, origin="lower", extent=[-np.pi, np.pi, -np.pi, np.pi])
    plt.tight_layout(1)
    pass
    

#Table of Contents
* [Introduction](#Introduction)
* [Topological invariants of Fermi surfaces](#Topological-invariants-of-Fermi-surfaces)
* [Graphene and protected Dirac cones](#Graphene-and-protected-Dirac-cones)
* [$d$-wave superconductors and edge states](#$d$-wave-superconductors-and-edge-states)
	* [Edge states](#Edge-states)
* [Weyl points](#Weyl-points)


**No content above this line is visible in edX**

# Introduction

Ashvin Vishwanath from the University of California, Berkeley will introduce Weyl semi-metals and other examples of gapless yet topological systems.

In [ ]:
MoocVideo("MAWwa4r1qIc", src_location='10.1-intro')

# Topological invariants of Fermi surfaces

The idea leading us to the topology of gapless systems is extremely simple, here it is:

> If we consider momentum as an external conserved parameter, we can study topological closings of the gap in momentum space.

Let's consider the simplest type of topological invariant, we've learned at the very beginning. Remember the simplest topological invariant of a 0D Hamiltonian, $\mathcal{Q} = \textrm{sign } H$, the number of filled states? What if we take two points in momentum space, $\mathbf{k}_1$ and $\mathbf{k}_2$, and consider a Hamiltonian with the number of filled states changing by $n$ between these two points? Obviously we can make a conclusion that there are at least $n$ Fermi surfaces lying on every path between $\mathbf{k}_1$ and $\mathbf{k}_2$ in momentum space.

Now we just need to take this idea and to apply it to more interesting systems and topological invariants!

What types of topological invariants can matter? Unless something very special happens, we cannot make use of time-reversal or particle-hole symmetries: in momentum space these only have an immediate effect in isolated $\mathbf{k}$-points, where every momentum component is either $0$ or $\pi$. So there doesn't exist a path in momentum space, which would have either of the symmetries effective in each point.

So we are left with only two symmetry classes: A and AIII (no symmetry at all or sublattice/chiral symmetry), and with only two invariants: if there is a sublattice symmetry, a winding number can be defined, and without it there's a Chern number.

# Graphene and protected Dirac cones

We've already analysed the 0D Chern number that stabilizes the usual Fermi surfaces. Let's go one dimension higher, and study winding numbers in systems with sublattice symmetry around 1D loops. 

For a winding number to be nonzero, we need to consider 1D loops in momentum space. So as a reminder, with sublattice symmetry the Hamiltonian can always be brought to the form

$$
H = \begin{pmatrix}
0 & h(\mathbf{k}) \\
h^\dagger(\mathbf{k}) & 0
\end{pmatrix}
$$

The topological invariant is a nonzero winding of $\det h(\mathbf{k})$ when $\mathbf{k}$ goes around some contour. Since $h(\mathbf{k})$ is continuous, it means that its determinant will also have to vanish somewhere inside this contour.

To study a particular example where this appears, let's return to graphene, which we studied as a simple limit of Haldane model.
For graphene we have

$$h(k_x, k_y) = t_1 e^{i k_x a_1} + t_2 e^{i k_x a_2} + t_3 e^{i k_x a_3},$$

where $t_1, t_2, t_3$ are the three hoppings connecting a site in one of the two graphene sublattices, and $a_1, a_2, a_3$ are the lattice vectors connecting one unit cell to its neighbors.

To consider something specific, let's take $t_2 = t_3 = t$ and vary $t_1$. This is how the band structure and $\det h$ look like:

In [ ]:
par = SimpleNamespace(t_1=1.0, t_23=1.0, kx=0.0, ky=0.0)
sys = graphene_infinite()

def graphene_plot(**kwargs):
    
    par.__dict__.update(kwargs)
    fig = plt.figure(figsize=(9, 3.5))
    
    ax0 = fig.add_subplot(122)
    dets = graphene_offdiag(sys, args=[par,], steps=80)
    plot_dets(dets)
    plot_format_deth(ax0)
    
    ax1 = fig.add_subplot(121, projection='3d')
    plot_dispersion(sys, par, ax=ax1)
    ax1.set_xlim([-np.pi, np.pi])
    ax1.set_xticks([-np.pi, np.pi])
    ax1.set_xticklabels([r'$-\pi$', r'$\pi$'])
    ax1.set_xlabel('$k_x$')
    ax1.set_ylim([-np.pi, np.pi])
    ax1.set_yticks([-np.pi, np.pi])
    ax1.set_yticklabels([r'$-\pi$', r'$\pi$'])
    ax1.set_ylabel('$k_y$')
    ax1.set_title(r'Graphene, $t_1 = %1.1f\times t$' % par.t_1)
    ax1.set_zlim3d(-5, 5)
    ax1.set_zlabel('$E$')
    ax1.set_zticks([])
    return fig

StaticInteract(lambda t_1: graphene_plot(t_1=(1 + 0.2*t_1)),
               t_1=RangeWidget(0, 7))

In the left panel you see that the band structure has gapless points. The right panel shows $\det h$ by using hue as a phase and intensity as magnitude (so white is $\det h = 0$). There are two Dirac points (you see 6, but this is since we plot more than one Brillouin zone). 

We also see that the winding numbers around these two Dirac points have opposite signs (because by going around them clockwise you encounter red, blue and green colors in opposite orders). This must always be the case since the winding number around the edges of the complete Brillouin zone must vanish - as you walk down every edge of the Brillouin zone twice in opposite directions, their contributions always cancel.

As $t_1$ increases, the two poles move towards each other, eventually annihilating and leaving a completely gapped dispersion relation. Let's now try to write an effective model for the dispersion at each pole and at the phase transition point.

We know that $\det h$ has to vanish next to some point $\mathbf{k}_0$. We can write expand it to a linear order next to this point, which immediately leaves us with a Hamiltonian

$$
H(\mathbf{k}) =
\begin{pmatrix}
0 & e^{i\alpha} (v_x \delta k_x + i v_y \delta k_y) \\
e^{-i\alpha} (v_x \delta k_x - i v_y \delta k_y) & 0
\end{pmatrix},
$$

where $\mathbf{\delta k}$ is of course the difference between $\mathbf{k}$ and the Dirac point momentum. Of course this is the 2D Dirac equation, which should be very familiar now.

At the phase transition where the two Dirac points annihilate, we can also quickly guess that the correct dispersion should be a quadratic function along the axis connecting the two Dirac points, and linear along the other axis (this is also what we see in the plot). So we then have:

$$
H(\mathbf{k}) =
\begin{pmatrix}
0 & e^{i\alpha} (\beta \delta k_1^2 + m + i v_2 \delta k_2) \\
e^{-i\alpha} (\beta \delta k_1^2 + m - i v_2 \delta k_2) & 0
\end{pmatrix},
$$

so that for $m>0$ we have a fully gapped Hamiltonian, and for $m<0$ there are two Dirac points.

# $d$-wave superconductors and edge states

The presence of gapless points with Dirac dispersion was known for quite some time before graphene. They exist in the cuprate family of high temperature superconductors, known to have a $d$-wave order parameter. These materials are layered, with weak couplings between the layer, so in the study of these complicated systems often one starts with a simplified two-dimensional Hamiltonian.

This Hamiltonian just has a usual kinetic term corresponding to a single particle band and a superconducting pairing proportional to $k_x^2 - k_y^2$, so together

$$
H = \begin{pmatrix}
k^2/2m -\mu & \Delta (k_x^2 - k_y^2) \\
\Delta (k_x^2 - k_y^2) & \mu-k^2/2m
\end{pmatrix}
$$

There is no spin-orbit coupling here, so the Hamiltonian has a spinless time-reversal symmetry $H = H^*$. It also has a particle-hole symmetry $H= - \tau_y H^* \tau_y$. Their product, the chiral symmetry $H = -\tau_y H \tau_y$ allows the Hamiltonian to have gapless points where both the single-particle dispersion and the pairing vanish.

## Difference between sublattice symmetries

The time-reversal symmetry ensures that the winding points come in pairs at opposite momenta, just like in graphene.
In graphene, however the chiral symmetry operator $\sigma_z$ commuted with the time-reversal symmetry, so there applying time-reversal symmetry changes the direction of a loop in momentum space, but leaves the winding number invariant. On the other hand in superconductors, $\tau_y$ is odd under time-reversal, and the winding is invariant under it.

This means that a Dirac point at momentum $k$ and positive winding must come together with a Dirac point at $-k$ and also positive winding. Since the total winding over the Brillouin zone must be 0, this means that in superconducting systems the Dirac points come in quadruplets: two with positive winding and two with negative winding.

The $d$-wave superconductor Hamiltonian gives just that: there are 4 Dirac points at $|k_x| = |k_y| = k_F / \sqrt{2}$.

In [ ]:
question = r"What happens is you make the 2D $d$-wave Hamiltonian 3D, by including a coupling between different layers?"

answers = ["The Dirac points couple and gap out.",
           "In 3D you cannot have a $d$-wave pairing.",
           "There will remain isolated gapless points in the larger 3D Brillouin zone.",
           "You get a closed 1D Dirac line of gap closings in the 3D Brillouin zone."]

explanation = (r"The real and imaginary parts of the solutions of $\det h(\mathbf{k})=0$ form two surfaces "
               r"in the Brillouin zone. The intersection of these two surfaces is a line.")

MoocMultipleChoiceAssessment(question=question, answers=answers, correct_answer=3, explanation=explanation)

## Edge states

Now let's see how bulk-edge correspondence can be made to work for gapless systems. We know that 

In [ ]:
par = SimpleNamespace(mu=2.0, t=1.0, delta=1.0, kx=0.0, ky=0.0)
sys0 = d_wave_ribbon(50, direction='topo')
sys1 = d_wave_ribbon(50, direction='triv')
sys2 = d_wave_infinite()


def d_wave_plot(sys0, sys1, sys2, args):
    fig = plt.figure(figsize=(10, 6))
    
    def plot_format_bands(ax):
        ax.set_color_cycle('b')
        ax.set_ylim([-2, 2])
        ax.set_yticks([-2, 0, 2])
        ax.set_yticklabels(['$-2$', '$0$', '$2$'])
        ax.set_ylabel('energy')
        ax.set_xticks([-np.pi, 0, np.pi])
        ax.set_xticklabels(['$-\pi$', '$0$', '$\pi$'])
        ax.set_xlabel('$k_x + k_y$')
    
    ax0 = fig.add_subplot(2,2,3)
    plot_format_bands(ax0)
    kwant.plotter.bands(sys0, args, ax=ax0, momenta=55)
    ax0.set_title('Ribbon with edge states')
    
    ax1 = fig.add_subplot(2,2,4)
    dets = d_wave_offdiag(sys2, args, steps=100)
    plot_dets(dets)
    plot_format_deth(ax1)
    
    ax2 = fig.add_subplot(2,2,1)
    plot_format_bands(ax2)
    kwant.plotter.bands(sys1, args, ax=ax2, momenta=55)
    ax2.set_title('Ribbon without edge states')
    
    ax3 = fig.add_subplot(2,2,2)
    plot_dets(dets)
    plot_format_deth(ax3)
    
    #draw some extra lines to show why we get edge states in one ribbon but not the other.
    k = np.arccos(1-par.mu/par.t/4)
    ax0.plot([0, 0], [-6, 6], 'k')
    ax0.plot([2*k, 2*k], [-6, 6], 'k--')
    ax0.plot([-2*k, -2*k], [-6, 6], 'k--')
    ax1.plot([-np.pi, np.pi], [np.pi, -np.pi], 'k')
    ax1.plot([-np.pi, np.pi-2*k], [np.pi-2*k, -np.pi], 'k--')
    ax1.plot([-np.pi+2*k, np.pi], [np.pi, -np.pi+2*k], 'k--')
    ax2.plot([k, k], [-6, 6], 'k--')
    ax2.plot([-k, -k], [-6, 6], 'k--')
    ax3.plot([-k, -k], [-np.pi, np.pi], 'k--')
    ax3.plot([k, k], [-np.pi, np.pi], 'k--')
    
d_wave_plot(sys0, sys1, sys2, args=[par,])

# Weyl points

So far we've seen two examples of Dirac points in two dimensions, the surface of a 3D topological insulator and graphene. You might wonder why don't we have such cones in three dimensions? These do indeed exist and are called Weyl points instead of Dirac points. The reason is historical - Dirac's equation for the electron (which  is in 3D) involved states with 4-components, two for the electron and two for the hole. The direct generalization of graphene that we will talk about involves two component electrons. This two component electrons with linear dispersion was studied first by Weyl and has strange properties as we will illustrate below.

Let us start by writing the low-energy Hamiltonian for the three dimensional generalization of graphene:

$$H({\bf k})=(\sigma_x k_x+\sigma_y k_y+\sigma_z k_z).$$

Here you might think of $\sigma_{x,y,z}$ as the spin of the electron (just as on the surface of a topological insulator).

Next we try the usual thing we would do with a two-dimensional Dirac cone - namely try to see what we get when we gap it out by applying a magnetic field $\bf\sigma\cdot B$. Adding such a term, we find that the Hamiltonian is transformed as 

$$H({\bf k})\rightarrow H({\bf k})+{\bf\sigma\cdot B}={\bf\sigma\cdot (k+B)}.$$

The key observation here is that the addition of a magnetic field effectively shifts the wave-vector as 

$${\bf k}\rightarrow \tilde{\bf k}={\bf k+ B}.$$

> So applying the most general perturbation we can think of does not gap the Weyl point where the energy vanishes, but rather only shifts it around in momentum space. This feels like some kind of topological protection.

In [ ]:
sys = Weyl_slab(w=15)
par = SimpleNamespace(t=1.0, mu=2.0)
def Weyl_plot(**kwargs):
    par.__dict__.update(kwargs)
    fig, ax = plot_dispersion(sys, par, kx=[-np.pi, 0], ky=[-np.pi, np.pi])
    ax.set_title('Weyl semimetal, $\mu = %1.1f$' % par.mu)
    ax.set_zlabel('$E$')
    ax.set_xticks([-np.pi, 0])
    ax.set_xticklabels(['$-\pi$', '$0$'])
    ax.set_xlabel('$k_x$')
    ax.set_yticks([-np.pi, np.pi])
    ax.set_yticklabels(['$-\pi$', '$\pi$'])
    ax.set_ylabel('$k_y$')
    ax.set_zticks([])
    return fig

StaticInteract(lambda mu: Weyl_plot(mu=0.4*mu), mu=RangeWidget(-1, 5))

Is there a sense in which Weyl points are "topological"? They are clearly protected - so is there some topological reason for it. As in the rest of this section, the topology of gapless system becomes apparent by looking at the Hamiltonian on lower dimensional subspaces of the momentum space. For the case of Weyl, the momentum space is three dimensional, so it is natural to try two dimensional subspaces of the momentum space.

The most natural subspace to choose is to fix $k_z=m$ the Weyl Hamiltonian becomes that of a massive 2D Dirac cone

$$H_{2D,Dirac}(k_x,k_y;m)\equiv H(k_x,k_y,k_z=m)=(\sigma_x k_x+\sigma_y k_y+m\sigma_z).$$

As we talked about in week 4, with Chern insulators, massive Dirac model had a Chern number, which would change by $1$ if $m$ changed sign. 

> So we can think of the Weyl Hamiltonian in the momentum planes for $k_z<0$ and $k_z>0$ as Chern insulators with Chern numbers $n_{Ch}=0$ (i.e. trivial) and $n_{Ch}=1$ (topological).  The Hamiltonian at $k_z=0$ is at the phase transition point of the Chern insulator, which supports a gapless Dirac point.

Systems with Weyl points are known as Weyl semi-metals. Just like other topological phases, have as an interesting surface spectrum. We can understand this easily by viewing the Weyl point as a stack of Chern insulators in momentum space. For any surface in a plane that contains the $z$-axis, we can treat $k_z$ as a conserved quantity. At this $k_z=m$, the Hamiltonian is just that of a Chern insulator with an appropriate Chern number. For the range of $k_z$ where the Chern number $n_{Ch}(k_z)=1$, the surface spectrum supports chiral edge states with an energy approximated at low energy by 

$$E(k_x,k_z)\approx v(k_z)k_x.$$

We can consider the edge states over a range of $k_z$ together to visualize the "surface states". 

> The unique property of the surface states is that if we set $k_x=0$ then the energy vanishes on a line in the surface spectrum. This line actually terminates at $k_z=0$, where the Chern number changes. Such lines that are referred to as "Fermi arcs", are the unique bounday properties (hence the bulk-boundary correspondence) for the Weyl semimetal.

At large enough $k_z$, the two dimensional Hamiltonian $H_{2D,Dirac}(k_x,k_y;k_z)$ becomes trivial i.e. $n_{Ch}(|k_z|\rightarrow \infty)=0$. This means that if the Chern number $n_{Ch}=1$ in a range of $k_z$, then $n_{Ch}(k_z)$ must change twice resulting in two Weyl points. So Weyl points come in pairs. These points map onto the ends of the Fermi arcs on the surface. 

In [ ]:
question = r"What protects the surface state of Weyl semi-metals from scattering inside the bulk Weyl point?"

answers = ["Chiral symmetry.",
           "The energy gap in the bulk.",
           "Absence of scattering.",
           "The non-zero Chern number of the bulk."]

explanation = (r"The bulk has gapless states due to the Weyl point. "
               "Only momentum conservation therefore protects surface states from going into the bulk.")

MoocMultipleChoiceAssessment(question=question, answers=answers, correct_answer=2, explanation=explanation)

**Questions about what you just learned? Ask them below!**

In [ ]:
MoocDiscussion("Questions", "Topology in gapless systems")